foo. 025

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import crs

import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

Define local file locations

In [6]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cli.030/"

file_name1 = "ai_yr.tif"
file_name2 = "pet_he_yr1"

local_orig1 = local_folder+ file_name1
local_orig2 = local_folder+ file_name2 + ".tif"  

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 
# local_edit2 = local_orig2[:-orig_extension_length] + "edit.tif" 

upload_name1 = "cli_030_global_aridity"
upload_name2 = "for_025_global_PET"


s3

In [7]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cli_030_global_land_aridity/"

s3_key_orig1 = s3_folder + upload_name1 + ".tif" 
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + upload_name2 + ".tif" 
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [8]:
files = [local_orig1, local_orig2]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

RasterioIOError: '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cli.030/ai_yr.tif' does not exist in the file system, and is not recognized as a supported dataset name. 

Upload orig and edit files to s3

In [9]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))

# # Edit
# s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
#                          Callback=ProgressPercentage(local_edit1))

FileNotFoundError: [WinError 2] The system cannot find the file specified: '/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cli.030/ai_yr.tif'

In [18]:
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_orig1
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_orig1
os.environ["Zs3_key2"] = "s3://wri-public-data/resourcewatch/raster/foo_025_global_PET/foo_025_global_PET.tif"
os.environ["Zgs_key2"] = "gs://resource-watch-public/resourcewatch/raster/foo_025_global_PET/foo_025_global_PET.tif"

In [19]:
!echo %Zs3_key%
!echo %Zgs_key%
!echo %Zs3_key2%
!echo %Zgs_key2%

s3://wri-public-data/resourcewatch/raster/cli_030_global_land_aridity/cli_030_global_aridity.tif
gs://resource-watch-public/resourcewatch/raster/cli_030_global_land_aridity/cli_030_global_aridity.tif
s3://wri-public-data/resourcewatch/raster/foo_025_global_PET/foo_025_global_PET.tif
gs://resource-watch-public/resourcewatch/raster/foo_025_global_PET/foo_025_global_PET.tif


In [20]:
!gsutil cp %Zs3_key2% %Zgs_key2%


Copying s3://wri-public-data/resourcewatch/raster/foo_025_global_PET/foo_025_global_PET.tif [Content-Type=image/tiff]...
/ [0 files][    0.0 B/193.2 MiB]                                                
-
- [0 files][  4.4 MiB/193.2 MiB]                                                
\
|
| [0 files][ 12.6 MiB/193.2 MiB]                                                
/
/ [0 files][ 21.1 MiB/193.2 MiB]                                                
-
\
\ [0 files][ 28.6 MiB/193.2 MiB]                                                
|
/
/ [0 files][ 37.6 MiB/193.2 MiB]                                                
-
- [0 files][ 46.2 MiB/193.2 MiB]                                                
\
|
| [0 files][ 53.6 MiB/193.2 MiB]                                                
/
-
- [0 files][ 61.6 MiB/193.2 MiB]                                                
\
\ [0 files][ 70.1 MiB/193.2 MiB]                                                
|
/
/ [0 files][ 78.9 MiB/193.2 MiB]     

In [ ]:
os.environ["asset_id"] = "users/resourcewatch/cli_030_global_aridity"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%

In [21]:
os.environ["asset_id2"] = "users/resourcewatch/foo_025_global_PET"
!earthengine upload image --nodata_value=-32768 --asset_id=%asset_id2% %Zgs_key2% -p Stats="Minimum=24.000, Maximum=2383.000, Mean=1062.422, StdDev=617.488"


Started upload task with ID: ZCIFSOQC2ZB6QONECABBRLX7


In [26]:
!earthengine task info ZCIFSOQC2ZB6QONECABBRLX7

ZCIFSOQC2ZB6QONECABBRLX7:
  State: RUNNING
  Type: Upload
  Description: Asset ingestion: users/resourcewatch/foo_025_global_PET
  Created: 2017-11-20 13:42:16.191000
  Started: 2017-11-20 13:42:20.426000
  Updated: 2017-11-20 13:52:20.570000


In [25]:
!earthengine acl set public users/resourcewatch/foo_025_global_PET 

Asset 'users/resourcewatch/foo_025_global_PET' does not exist or doesn't allow this operation.
